<a href="https://colab.research.google.com/github/glee255/fetch/blob/main/Fetch_HomeTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Brands Table**
 - Read json file.
 - Read id values within the dictionaries.

In [1]:
# Read json file
import pandas as pd
import json

with open('/content/brands.json', 'r') as file:
    json_text = file.read()

json_objects = json_text.strip().split('\n')
data = [json.loads(obj) for obj in json_objects]
brands = pd.DataFrame(data)

In [2]:
# Read  id values
def extract(d):
    try:
        if "$oid" in d:
            return d["$oid"]
        elif "$id" in d and "$oid" in d["$id"]:
            return d["$id"]["$oid"]
        else:
            return None
    except (KeyError, TypeError):
        return None

brands['brand_id'] = brands['_id'].apply(lambda x: extract(x))
brands['cpg_id'] = brands['cpg'].apply(lambda x: extract(x))

In [3]:
# 1167 brands in total
brands.shape

(1167, 10)

### Missing values
 - There seems to be too many missing values in category, categoryCode, topBrand, brandCode.
 - Removing null values may remove the entire data set. Replacing null values does not make sense for this case.

In [4]:
brands.isna().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
brand_id          0
cpg_id            0
dtype: int64

### Misleading column name
 - categoryCode seems to have text values and not categorical values.
 - Also it almost seems same as the category values.

In [5]:
brands['categoryCode'].value_counts()

BAKING                           359
CANDY_AND_SWEETS                  71
BEER_WINE_SPIRITS                 31
HEALTHY_AND_WELLNESS              14
GROCERY                           11
BABY                               7
CLEANING_AND_HOME_IMPROVEMENT      6
BREAD_AND_BAKERY                   5
DAIRY_AND_REFRIGERATED             5
PERSONAL_CARE                      4
BEVERAGES                          1
OUTDOOR                            1
MAGAZINES                          1
FROZEN                             1
Name: categoryCode, dtype: int64

In [6]:
brands['category'].value_counts()

Baking                         369
Beer Wine Spirits               90
Snacks                          75
Candy & Sweets                  71
Beverages                       63
Magazines                       44
Health & Wellness               44
Breakfast & Cereal              40
Grocery                         39
Dairy                           33
Condiments & Sauces             27
Frozen                          24
Personal Care                   20
Baby                            18
Canned Goods & Soups            12
Beauty                           9
Cleaning & Home Improvement      6
Deli                             6
Beauty & Personal Care           6
Household                        5
Bread & Bakery                   5
Dairy & Refrigerated             5
Outdoor                          1
Name: category, dtype: int64

### Inconsistent values
 - brandCode contains inconsistent values such as numeric, text and numeric and text combined.

In [7]:
brands['brandCode'].value_counts()

                                 35
GOODNITES                         2
HUGGIES                           2
ROYAL DANSK                       1
SOL                               1
                                 ..
TEST BRANDCODE @1599159969028     1
TEST BRANDCODE @1597350074404     1
SEDAL                             1
RED ROCK DELI                     1
TEST BRANDCODE @1613158231644     1
Name: brandCode, Length: 897, dtype: int64

### Drop irrelevant columns
 - Remove '_id', 'cpg', 'categoryCode columns.
 - 'cpg' value needs to be checked.

In [8]:
# Remove '_id','cpg','categoryCode. 'cpg' value needs to be checked.
colsToDrop = ['_id','categoryCode','cpg']
brands.drop(colsToDrop, axis=1, inplace=True)

### Check duplicates
- No duplicates

In [9]:
# No duplicates
brands.duplicated().any()

False

In [10]:
# Check the first five rows
brands.head()

,barcode,category,name,topBrand,brandCode,brand_id,cpg_id
0,511111019862,Baking,test brand @1612366101024,False,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550
1,511111519928,Beverages,Starbucks,False,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba
2,511111819905,Baking,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559
3,511111519874,Baking,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559
4,511111319917,Candy & Sweets,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7


### Sqlite3 table conversion

In [11]:
file_name = 'brands.xlsx'
brands.to_excel(file_name, index=False)

In [12]:
for column in brands.select_dtypes(include=['float64']).columns:
    brands[column] = brands[column].fillna(0).astype(int)

for column in brands.select_dtypes(include=['object']).columns:
    brands[column] = brands[column].astype(str)

In [13]:
import sqlite3

conn = sqlite3.connect('fetch.db')
cur = conn.cursor()

brands.to_sql('brands', conn, if_exists='replace', index=False)

1167

### **Receipts Table**
 - Read json file.
 - Retrieve the key values from rewardsReceiptItemList column. Twelve key values contained.
 - Retrieve id values within the dictionary.
 - Retrieve and convert dates from date columns

In [14]:
# Read json file
with open('/content/receipts.json', 'r') as file:
    json_text = file.read()

json_objects = json_text.strip().split('\n')
data1 = [json.loads(obj) for obj in json_objects]
receipts = pd.DataFrame(data1)

In [15]:
# Extract the key values from rewardsReceiptItemList column. Too many key values contained.
def extract1(list_of_values, key):
    if not isinstance(list_of_values, list):
        return ''
    values = [str(d.get(key)) for d in list_of_values if d.get(key) is not None]
    return ','.join(values)

receipts['barcodes'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'barcode'))
receipts['description'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'description'))
receipts['finalPrice'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'finalPrice'))
receipts['itemPrice'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'itemPrice'))
receipts['needsFetchReview'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'needsFetchReview'))
receipts['partnerItemId'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'partnerItemId'))
receipts['preventTargetGapPoints'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'preventTargetGapPoints'))
receipts['quantityPurchased'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'quantityPurchased'))
receipts['userFlaggedBarcode'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'userFlaggedBarcode'))
receipts['userFlaggedNewItem'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'userFlaggedNewItem'))
receipts['userFlaggedPrice'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'userFlaggedPrice'))
receipts['userFlaggedQuantity'] = receipts['rewardsReceiptItemList'].apply(lambda x: extract1(x, 'userFlaggedQuantity'))
receipts['receipt_id'] = receipts['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else None)

In [16]:
# Extract dates from date columns
def extract2(d):
    try:
        oid_value = d['$date']
        return oid_value
    except (KeyError, TypeError):
        return None

receipts['create_d'] = receipts['createDate'].apply(lambda x: extract2(x))
receipts['scan_d'] = receipts['dateScanned'].apply(lambda x: extract2(x))
receipts['finish_d'] = receipts['finishedDate'].apply(lambda x: extract2(x))
receipts['modify_d'] = receipts['modifyDate'].apply(lambda x: extract2(x))
receipts['point_d'] = receipts['pointsAwardedDate'].apply(lambda x: extract2(x))
receipts['purchase_d'] = receipts['purchaseDate'].apply(lambda x: extract2(x))

In [17]:
# Convert data to year and month
from datetime import datetime
receipts['create_ym'] = receipts['create_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['scan_ym'] = receipts['scan_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['finish_ym'] = receipts['finish_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['modify_ym'] = receipts['modify_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['point_ym'] = receipts['point_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
receipts['purchase_ym'] = receipts['purchase_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)

### Drop irrelevant columns

- Remove json formatted columns and columns that were used for retrieving and conversion.



In [18]:
# Remove '_id','createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate'
colsToDrop = ['rewardsReceiptItemList','_id','create_d','scan_d','finish_d','modify_d','point_d','purchase_d','createDate','dateScanned','finishedDate','modifyDate','pointsAwardedDate','purchaseDate']
receipts.drop(colsToDrop, axis=1, inplace=True)

In [19]:
receipts.shape

(1119, 26)

### Missing values
 - Customers may not earn bonus points. Null value reasonable.
 - But other than the bonus points related columns, other needs to be checked.
 - Barcodes column contatins empty string that is not recognized as na value.

In [20]:
receipts.isna().sum()

bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
barcodes                     0
description                  0
finalPrice                   0
itemPrice                    0
needsFetchReview             0
partnerItemId                0
preventTargetGapPoints       0
quantityPurchased            0
userFlaggedBarcode           0
userFlaggedNewItem           0
userFlaggedPrice             0
userFlaggedQuantity          0
receipt_id                   0
create_ym                    0
scan_ym                      0
finish_ym                  551
modify_ym                    0
point_ym                   582
purchase_ym                448
dtype: int64

In [21]:
# empty string not recognized as na value
receipts['barcodes'].head()

0                 4011
1    4011,028400642255
2                     
3                 4011
4            4011,1234
Name: barcodes, dtype: object

In [22]:
receipts.head(5)

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,barcodes,description,finalPrice,...,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,receipt_id,create_ym,scan_ym,finish_ym,modify_ym,point_ym,purchase_ym
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,4011,ITEM NOT FOUND,26.00,...,True,26.00,5,5ff1e1eb0a720f0523000575,2021-01,2021-01,2021-01,2021-01,2021-01,2021-01
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,"4011,028400642255","ITEM NOT FOUND,DORITOS TORTILLA CHIP SPICY SWE...","1,10.00",...,True,10.00,1,5ff1e1bb0a720f052300056b,2021-01,2021-01,2021-01,2021-01,2021-01,2021-01
2,5.0,All-receipts receipt bonus,5,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,,,,...,True,26.00,3,5ff1e1f10a720f052300057a,2021-01,2021-01,None,2021-01,None,2021-01
3,5.0,All-receipts receipt bonus,5.0,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,4011,ITEM NOT FOUND,28.00,...,True,28.00,4,5ff1e1ee0a7214ada100056f,2021-01,2021-01,2021-01,2021-01,2021-01,2021-01
4,5.0,All-receipts receipt bonus,5.0,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,"4011,1234",ITEM NOT FOUND,"1,2.56",...,True,2.56,3,5ff1e1d20a7214ada1000561,2021-01,2021-01,2021-01,2021-01,2021-01,2021-01


### Data issues
 - Empty string not recognized as na value
 - Multiple values in barcodes, finalPrice, itemPrice, partnerItemId, quantityPurchased columns. First form normalization needed.
 - Split the barcodes to use as foreign key.
 - pointsEarned, totalSpent, finalPrice, itemPrice, quantityPurchased data types are object. Data type need to be changed to numeric.

In [23]:
import numpy as np
receipts['barcodes'] = receipts['barcodes'].replace(r'^\s*$', np.nan, regex=True)

In [24]:
receipts['barcodes'] = receipts['barcodes'].str.split(',')
receipts = receipts.explode('barcodes')
receipts.rename(columns={'barcodes': 'barcode'}, inplace=True)
receipts['barcode'] = receipts['barcode'].str.strip()

In [25]:
to_convert = ['pointsEarned', 'totalSpent', 'finalPrice', 'itemPrice', 'quantityPurchased']

# Convert each column to numeric, coercing errors to NaN
for column in to_convert:
    receipts[column] = pd.to_numeric(receipts[column], errors='coerce')

In [26]:
receipts.shape

(3616, 26)

In [27]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3616 entries, 0 to 1118
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        2709 non-null   float64
 1   bonusPointsEarnedReason  2709 non-null   object 
 2   pointsEarned             2972 non-null   float64
 3   purchasedItemCount       3132 non-null   float64
 4   rewardsReceiptStatus     3616 non-null   object 
 5   totalSpent               3181 non-null   float64
 6   userId                   3616 non-null   object 
 7   barcode                  3090 non-null   object 
 8   description              3616 non-null   object 
 9   finalPrice               433 non-null    float64
 10  itemPrice                433 non-null    float64
 11  needsFetchReview         3616 non-null   object 
 12  partnerItemId            3616 non-null   object 
 13  preventTargetGapPoints   3616 non-null   object 
 14  quantityPurchased       

In [28]:
receipts.isna().sum()

bonusPointsEarned           907
bonusPointsEarnedReason     907
pointsEarned                644
purchasedItemCount          484
rewardsReceiptStatus          0
totalSpent                  435
userId                        0
barcode                     526
description                   0
finalPrice                 3183
itemPrice                  3183
needsFetchReview              0
partnerItemId                 0
preventTargetGapPoints        0
quantityPurchased          3183
userFlaggedBarcode            0
userFlaggedNewItem            0
userFlaggedPrice              0
userFlaggedQuantity           0
receipt_id                    0
create_ym                     0
scan_ym                       0
finish_ym                   797
modify_ym                     0
point_ym                    811
purchase_ym                 448
dtype: int64

### Sqlite3 table conversion

In [29]:
for column in receipts.select_dtypes(include=['float64']).columns:
    receipts[column] = receipts[column].fillna(0).astype(int)

In [30]:
receipts.to_sql('receipts', conn, if_exists='replace', index=False)

3616

In [31]:
file_name = 'receipts.xlsx'
receipts.to_excel(file_name, index=False)

### **Users Table**
 - Read json file.
 - Retrieve id values within the oid dictionary.
 - Retrieve and convert dates from date dictionary.

In [32]:
with open('/content/users.json', 'r') as file:
    json_text = file.read()

json_objects = json_text.strip().split('\n')
data2 = [json.loads(obj) for obj in json_objects]
users = pd.DataFrame(data2)

In [33]:
users['create_d'] = users['createdDate'].apply(lambda x: extract2(x))
users['login_d'] = users['lastLogin'].apply(lambda x: extract2(x))
users['user_id'] = users['_id'].apply(lambda x: x.get('$oid') if isinstance(x, dict) else None)

In [34]:
users['create_ym'] = users['create_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)
users['scan_ym'] = users['login_d'].apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m') if pd.notnull(x) else None)

### Missing values
 - Missing values in lastLogin, signUpSource, state and scan_ym columns.

In [35]:
users.shape

(495, 12)

In [36]:
users.isna().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
create_d         0
login_d         62
user_id          0
create_ym        0
scan_ym         62
dtype: int64

### Consistent Values
 - Values are consistent across categorical or text columns

In [37]:
users['signUpSource'].value_counts()

Email     443
Google      4
Name: signUpSource, dtype: int64

In [38]:
users['role'].value_counts()

consumer       413
fetch-staff     82
Name: role, dtype: int64

In [39]:
users['state'].value_counts()

WI    396
NH     20
AL     12
OH      5
IL      3
KY      1
CO      1
SC      1
Name: state, dtype: int64

In [40]:
users['active'].value_counts()

True     494
False      1
Name: active, dtype: int64

### Remove irrelevant columns
 -  Remove json formatted columns and columns that were used for retrieving and conversion.

In [41]:
colsToDrop = ['_id','createdDate','lastLogin']
users.drop(colsToDrop, axis=1, inplace=True)

### Check duplicates
 - Within the users table, 283 users are duplicated in the 495 data points in the table.
 - Drop the 283 duplicated users.

In [42]:
users.duplicated().any()

True

In [43]:
users['user_id'].duplicated().sum()

283

In [44]:
users = users.drop_duplicates()

### Sqlite3 table conversion

In [ ]:
for column in users.select_dtypes(include=['float64']).columns:
    users[column] = users[column].fillna(0).astype(int)

In [46]:
users.to_sql('users', conn, if_exists='replace', index=False)

212

In [47]:
file_name = 'users.xlsx'
users.to_excel(file_name, index=False)

### **Business questions - TOP 5 brands**

- What are the top 5 brands by receipts scanned for most recent month?
  - The most recent month in the receipts table is March, 2021.
  - However, due to missing barcode values in receipt table or non listed barcode values in the brand table, most of them have null values. Receipts scanned in Jan, 2021 partially contain brand name values.
  
- How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
  - Top 5 brands for Jan 2021 are as below (containing name values)
    1. Tostitos
    2. Swanson
    3. Cracker Barrel Cheese
    4. Prego
    5. Diet Chris Cola


In [48]:
cur.execute("""
SELECT scan_ym,
       COUNT(receipt_id)
FROM receipts
GROUP BY scan_ym
ORDER BY scan_ym DESC
""")

rows = cur.fetchall()
for row in rows:
    print(row)

('2021-03', 43)
('2021-02', 500)
('2021-01', 3043)
('2020-11', 24)
('2020-10', 6)


In [49]:
cur.execute("""
SELECT
       A.scan_ym,
       B.name,
       COUNT(A.receipt_id) AS scanned,
       ROW_NUMBER() OVER (PARTITION BY scan_ym ORDER BY COUNT(DISTINCT A.receipt_id) DESC) AS RANK
FROM RECEIPTS A LEFT OUTER JOIN BRANDS B
ON A.barcode=B.barcode
GROUP BY A.scan_ym, B.name
ORDER BY scan_ym DESC
""")

rows = cur.fetchall()
for row in rows:
    print(row)

('2021-03', None, 43, 1)
('2021-02', None, 500, 1)
('2021-01', None, 2961, 1)
('2021-01', 'Tostitos', 23, 2)
('2021-01', 'Swanson', 11, 3)
('2021-01', 'Cracker Barrel Cheese', 10, 4)
('2021-01', 'Prego', 7, 5)
('2021-01', 'Diet Chris Cola', 7, 6)
('2021-01', 'Quaker', 3, 7)
('2021-01', 'Kraft', 3, 8)
('2021-01', 'Kettle Brand', 3, 9)
('2021-01', 'Jell-O', 3, 10)
('2021-01', 'V8', 4, 11)
('2021-01', 'Rice A Roni', 3, 12)
('2021-01', 'Pepperidge Farm', 5, 13)
('2021-01', 'Cheetos', 3, 14)
('2021-01', 'Sargento® Cheese', 1, 15)
('2021-01', 'Pacific Foods', 1, 16)
('2021-01', 'Mountain Dew', 1, 17)
('2021-01', 'Grey Poupon', 1, 18)
('2020-11', None, 24, 1)
('2020-10', None, 6, 1)


###**Business Questions - Spending and items purchased by rewardsReceiptStatus**
- When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
  - First of all, there is no value named 'Accepted'.
  - If comparing with 'Finished' status, average spending is 913 dollars which is approx. 33 times higher than 'Rejected' average spending of 27 dollars. (Assuming spending in 1 dollar unit.)
- When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
  - If comparing with 'Finished' status, total number of items purchased is 491,828 which is approx. 1,927 times higher than 'Rejected' total number of items purchased of 255.


In [50]:
cur.execute("""

    SELECT
        rewardsReceiptStatus,
        ROUND(AVG(CASE WHEN totalSpent IS NOT NULL THEN totalSpent END), 0) AS avg_totalSpent,
        SUM(CASE WHEN purchasedItemCount IS NOT NULL THEN purchasedItemCount ELSE 0 END) AS total_purchasedItemCount
    FROM
        receipts
    GROUP BY
        rewardsReceiptStatus
""")

rows = cur.fetchall()
for row in rows:
    print(row)

('FINISHED', 913.0, 491828)
('FLAGGED', 2064.0, 70063)
('PENDING', 27.0, 0)
('REJECTED', 27.0, 255)
('SUBMITTED', 0.0, 0)


In [51]:
cur.execute("""

WITH StatusTotals AS (
    SELECT
        rewardsReceiptStatus,
        ROUND(AVG(CASE WHEN totalSpent IS NOT NULL THEN totalSpent END), 0) AS avg_totalSpent,
        SUM(CASE WHEN purchasedItemCount IS NOT NULL THEN purchasedItemCount ELSE 0 END) AS total_purchasedItemCount
    FROM
        receipts
    GROUP BY
        rewardsReceiptStatus
)
SELECT
    st1.rewardsReceiptStatus AS status1,
    st2.rewardsReceiptStatus AS status2,
    ROUND((st1.avg_totalSpent - st2.avg_totalSpent) / st2.avg_totalSpent ) AS percent_change_avg_totalSpent,
    ROUND((st1.total_purchasedItemCount - st2.total_purchasedItemCount) / st2.total_purchasedItemCount ) AS percent_change_total_purchasedItemCount
FROM
    StatusTotals st1
JOIN
    StatusTotals st2 ON st1.rewardsReceiptStatus < st2.rewardsReceiptStatus

""")

rows = cur.fetchall()
for row in rows:
    print(row)


('FINISHED', 'FLAGGED', -1.0, 6.0)
('FINISHED', 'PENDING', 33.0, None)
('FINISHED', 'REJECTED', 33.0, 1927.0)
('FINISHED', 'SUBMITTED', None, None)
('FLAGGED', 'PENDING', 75.0, None)
('FLAGGED', 'REJECTED', 75.0, 273.0)
('FLAGGED', 'SUBMITTED', None, None)
('PENDING', 'REJECTED', 0.0, -1.0)
('PENDING', 'SUBMITTED', None, None)
('REJECTED', 'SUBMITTED', None, None)


### **Business Questions - Most recent users spending and transactions**
- Which brand has the most spend among users who were created within the past 6 months?
    - Tostitos has the most spending of 15,785 dollars among users who were created within the past 6 months.
- Which brand has the most transactions among users who were created within the past 6 months?
    - Tostitos has the most transactions of 23 among users who were created within the past 6 months.

- Please refer that the create_ym has missing month/year values of  Oct/2020 and Sep/2020 which needs to be included in the past 6 months window. As a result, it only add up 4 months cumulative values.

In [52]:
cur.execute("""

SELECT create_ym, count(*)
FROM users
group by create_ym
order by create_ym desc;

""")

# Fetch and print the results of the SELECT query
rows = cur.fetchall()
for row in rows:
    print(row)

('2021-02', 30)
('2021-01', 170)
('2020-12', 1)
('2020-11', 4)
('2020-07', 1)
('2020-01', 1)
('2017-12', 1)
('2017-09', 1)
('2017-07', 1)
('2015-04', 1)
('2014-12', 1)


In [53]:
cur.execute("""
WITH threshold AS (
    SELECT strftime('%Y-%m', MAX(create_ym || '-01'), '-6 months') AS threshold
    FROM users
)
SELECT A.name,
       ROUND(SUM(B.totalSpent), 0) AS "total_spend",
       COUNT(B.receipt_id) AS "transaction"
FROM brands A
INNER JOIN receipts B ON A.barcode = B.barcode
INNER JOIN users C ON B.userId = C.user_id,
threshold T
WHERE A.name IS NOT NULL
AND C.create_ym >= T.threshold
GROUP BY A.name
ORDER BY "total_spend" DESC;
""")

# Fetch and print the results of the SELECT query
rows = cur.fetchall()
for row in rows:
    print(row)

('Tostitos', 15785.0, 23)
('Pepperidge Farm', 14163.0, 3)
('V8', 9442.0, 2)
('Prego', 9442.0, 2)
('Diet Chris Cola', 9442.0, 2)
('Swanson', 7180.0, 11)
('Cracker Barrel Cheese', 4884.0, 2)
('Jell-O', 4753.0, 2)
('Cheetos', 4721.0, 1)
('Kettle Brand', 2399.0, 3)
('Grey Poupon', 743.0, 1)
('Quaker', 32.0, 1)
